# Separate annotations and reviews from database

In [1]:
import os
import rootpath
import json
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
root = rootpath.detect()
root

'/home/yiyi/Documents/masterarbeit/CPD'

In [3]:
db_path = '/home/yiyi/Documents/masterarbeit/cpd_annotations/api-json-server/db.json'

In [4]:
with open(db_path) as file:
    db = json.load(file)

In [5]:
annotations = db['annotations']

In [12]:
with open('data_processed/annotations.json', 'w') as file:
    json.dump(annotations, file)

In [13]:
reviews = db['reviews']

In [14]:
with open('data_processed/reviews.json', 'w') as file:
    json.dump(reviews, file)

## Get annotations about the aspect/sentiment.

In [15]:
annotations[0]

{'id': 'lZqrFrJub',
 'date': '2020-06-30T10:59:09.303Z',
 'filename': 'renovation_room_147',
 'fileId': '252',
 'username': 'Edita',
 'annotation': [{'id': 0, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 1, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 2, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 3, 'yes_sentiment': True},
  {'id': 4, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 5, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 6, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 7, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 8, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 9, 'yes_aspect': False, 'yes_sentiment': True}],
 'cptAnswer': {}}

In [6]:

new_annotations = defaultdict(list)
for a in annotations:
    idx = int(a['fileId'])
    username= a['username']
    if username !='yiyi':
        new_annotations[idx].append(a)
        

In [8]:
new_annotations[0]

[{'id': 'XrpM542r4',
  'date': '2020-06-30T11:24:40.059Z',
  'filename': 'facility_260',
  'fileId': '0',
  'username': 'Oana',
  'annotation': [{'id': 0, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 2, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 3, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 4, 'yes_aspect': True},
   {'id': 5, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 6, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 7, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 8, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 9, 'yes_aspect': True},
   {'id': 10, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 11, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 12, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 13, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 14, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 15, 'yes_aspect': False, 'yes_sentiment': True},
   {'id': 17, 'yes_aspect': True, 'yes_sen

In [9]:
annotations_dict = defaultdict(list)
for idx, al in new_annotations.items():
    adict = defaultdict(list)
    for a in al:
        username=a['username']
        adict[username].append(a)
    for username, a in adict.items():
        if len(a)>1:
            sorted_a = sorted(a, key=lambda i:i['date'])
            annotations_dict[idx].append(sorted_a[-1])
        else:
            annotations_dict[idx].append(a[0])

In [10]:
annotations_dict[1]

[{'id': 'jm6oOFqc9',
  'date': '2020-06-30T11:36:22.478Z',
  'filename': 'pool_260',
  'fileId': '1',
  'username': 'Oana',
  'annotation': [{'id': 0, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 1, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 2, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 3, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 4, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 5, 'yes_aspect': False, 'yes_sentiment': True},
   {'id': 7, 'yes_aspect': True},
   {'id': 8, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 9, 'yes_aspect': True},
   {'id': 10, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 11, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 12, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 13, 'yes_aspect': True, 'yes_sentiment': True},
   {'id': 14, 'yes_sentiment': True},
   {'id': 15, 'yes_sentiment': True},
   {'id': 18, 'yes_aspect': True, 'yes_sentiment': True}],
  'cptAnswer': {'0': 'c'}},

In [11]:
categories =['yes_aspect', 'yes_sentiment', 'no_aspect', 'no_sentiment']

In [71]:
def df_dict(annotations_dict):
    # idx, sentID, no_aspect, no_sentiment, yes_two
    dataframe_dict = dict()
    cpts_dict = dict()
    
    for idx, alist in annotations_dict.items():

        for a in alist:
            annotation=a['annotation']
            
            if a['cptAnswer']:
                cpts = a['cptAnswer']
                
                for nr, cpt in cpts.items():
                    id_nr = str(idx)+'_'+nr
                    
                    if id_nr not in cpts_dict:
                        cpts_dict[id_nr] = defaultdict(int)
                    cpts_dict[id_nr][cpt]+=1
                
                
            
            for anno in annotation:
                id_ = anno['id']
                
                yes_aspect_ = anno.get('yes_aspect', False)
                yes_sentiment_ = anno.get('yes_sentiment', False)

                id_sentid = str(idx)+'_'+str(id_)
                
                if id_sentid not in dataframe_dict:
                    dataframe_dict[id_sentid]={
                        'no-aspect':0,
                        'no-sentiment':0,
                        'yes-two':0
                    }
                    
                if not yes_aspect_:
                    dataframe_dict[id_sentid]['no-aspect'] += 1
                if not yes_sentiment_:
                    dataframe_dict[id_sentid]['no-sentiment'] += 1
                if yes_aspect_ and yes_sentiment_:
                    dataframe_dict[id_sentid]['yes-two'] += 1
                    
    return dataframe_dict, cpts_dict


In [94]:
dfdict, cpts_dict = df_dict(annotations_dict)

In [95]:
cpts_df = pd.DataFrame.from_dict(cpts_dict, orient='index')

In [96]:
cpts_df.head()

,a,b,c
85_0,2.0,NaN,NaN
42_0,2.0,NaN,NaN
86_0,2.0,NaN,NaN
253_0,2.0,NaN,NaN
87_0,1.0,NaN,1.0


In [99]:
cpts_df = cpts_df.fillna(0)

In [101]:
columns = cpts_df.columns

In [102]:
columns

Index(['a', 'b', 'c'], dtype='object')

In [105]:
cpts_df[columns]= cpts_df[columns].applymap(np.int64)

In [106]:
cpts_df

,a,b,c
85_0,2,0,0
42_0,2,0,0
86_0,2,0,0
253_0,2,0,0
87_0,1,0,1
...,...,...,...
78_0,0,0,2
419_1,0,0,1
242_1,0,0,1
80_1,0,0,2


In [110]:
total_col = cpts_df.sum(axis=1)

In [111]:
total_col

85_0     2
42_0     2
86_0     2
253_0    2
87_0     2
        ..
78_0     2
419_1    1
242_1    1
80_1     2
295_1    1
Length: 584, dtype: int64

In [112]:
cpts_df['Total']= total_col

In [113]:
indices = cpts_df.index

In [114]:
fileId, cpt_id = [], []
for id_ in indices:
    fileid, cptid = id_.split('_')
    fileId.append(int(fileid))
    cpt_id.append(int(cptid))
    

In [115]:
cpts_df['fileId']=fileId
cpts_df['cptId'] = cpt_id

In [116]:
sorted_cpts_df = cpts_df.sort_values(by=['fileId', 'cptId'])

In [117]:
sorted_cpts_df

,a,b,c,Total,fileId,cptId
0_0,1,0,1,2,0,0
1_0,0,0,2,2,1,0
2_0,1,0,1,2,2,0
3_0,1,0,1,2,3,0
3_1,1,0,1,2,3,1
...,...,...,...,...,...,...
417_0,1,0,0,1,417,0
418_0,1,0,0,1,418,0
418_1,0,0,1,1,418,1
419_0,1,0,0,1,419,0


In [91]:
sorted_cpts_df.head(20)

,a,b,c,fileId,cptId
0_0,1.0,NaN,1.0,0,0
1_0,NaN,NaN,2.0,1,0
2_0,1.0,NaN,1.0,2,0
3_0,1.0,NaN,1.0,3,0
3_1,1.0,NaN,1.0,3,1
3_2,NaN,NaN,2.0,3,2
3_3,NaN,NaN,2.0,3,3
3_4,NaN,NaN,2.0,3,4
4_0,1.0,NaN,1.0,4,0
4_1,NaN,NaN,2.0,4,1


In [ ]:
cpts_dict

In [ ]:
new_annotations[1]

In [ ]:
stas = defaultdict(int)
count = 0 
count2 = 0
for idx in sorted(new_annotations):
    a = new_annotations[idx]
    if len(a)==1:
        count+=1
    if len(a)>2:
        count2 +=1
        print(idx)
        print(a)

In [17]:
len(new_annotations)

419

In [16]:
count

108

In [20]:
count2

13

In [ ]:
new_annotations[1]

In [9]:
annotations[3]

{'id': 'rPrwxegEs',
 'date': '2020-06-30T10:59:16.025Z',
 'filename': 'facility_174',
 'fileId': '85',
 'username': 'Andrei',
 'annotation': [{'id': 0, 'yes_aspect': False},
  {'id': 1, 'yes_sentiment': True},
  {'id': 2},
  {'id': 3, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 4, 'yes_sentiment': True},
  {'id': 5},
  {'id': 6},
  {'id': 7, 'yes_sentiment': True},
  {'id': 8, 'yes_sentiment': True},
  {'id': 9, 'yes_sentiment': True},
  {'id': 10, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 11, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 12, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 13, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 14, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 15, 'yes_sentiment': True},
  {'id': 16, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 17, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 18, 'yes_aspect': True, 'yes_sentiment': True},
  {'id': 19, 'yes_aspect': True, 'yes_sentiment': True}],
